In [1]:
import matplotlib.pyplot as plt
import skimage
from skimage.io import imread
import numpy as np
from pathlib import Path

In [2]:
def image_combination(over, under):
    over = over/255
    under = under/255
    over = skimage.color.rgb2ycbcr(over)
    under = skimage.color.rgb2ycbcr(under)
    #overexposed
    Yfilt = skimage.restoration.denoise_bilateral(over[:,:,0], sigma_spatial=3)
    Yover = over[:,:,0] - Yfilt
    #underexposed
    Yunder = skimage.restoration.denoise_bilateral(under[:,:,0], sigma_spatial=3)
    #combine
    x = len(over[:,1])
    y = len(over[1,:])
    final = np.empty((x,y,3))
    final[:,:,0] = Yover + Yunder
    final[:,:,1] = over[:,:,1]
    final[:,:,2] = over[:,:,2]
    final_image = skimage.color.ycbcr2rgb(final)
    return final_image

In [3]:
def local_gamma(im):
    im =im/255
    ycbcr = skimage.color.rgb2ycbcr(im)
    y = ycbcr[:,:,0]
    y_filt = skimage.restoration.denoise_bilateral(y, sigma_spatial=5)
    mask = 255-y_filt
    row = np.size(y[:,1])
    col = np.size(y[1,:])
    Y_final = np.empty((row, col))
    for i in range(row):
        for j in range(col):
            exp2 = (128-mask[i,j])/128
            exp = 2**exp2
            value=(y[i,j]/255)**exp
            output=255*value
            Y_final[i,j]= output
    ycbcr[:,:,0] = Y_final
    final = skimage.color.ycbcr2rgb(ycbcr)
    return final

In [4]:
from mtcnn.mtcnn import MTCNN
detector = MTCNN()


In [53]:
from tqdm import tqdm
path = "../../../dataset/train/"

train_images_path = list(Path(path).rglob(f"*.jpg"))
values = []
for path in tqdm(train_images_path):
    im = imread(path)
    v = np.sum(im<128) / np.sum(im>128)
    values += [v]
values.sort()

100%|██████████| 11220/11220 [00:20<00:00, 536.28it/s] 


In [56]:
v = np.array(values)
print(np.sum(v>4))
print(np.sum(v<0.2))

288
127


In [149]:
def fix_image(im, count):
    faces = detector.detect_faces(im)
    if len(faces) == 0:
        value = np.sum(im<128) / np.sum(im>128)
        if value > 4 or value == "inf" and count <= 1:
            im = skimage.exposure.adjust_gamma(im, 0.5)
            c = count +1
            crop = fix_image(im, c)
            return crop
        elif value < 0.2 and count <= 1:
            im = skimage.exposure.adjust_gamma(im, 1.5)
            c = count +1
            crop = fix_image(im, c)
            return crop
        else:
            print("no faces")
            return None
    [X,Y,W,H] = faces[0]['box']
    crop = im[Y-10:Y+H+10, X-10:X+W+10]
    #crop = fix_zeros(crop)
    faces2 = detector.detect_faces(crop)
    if len(faces2) == 0:
        print("no faces")
        return None
    b=True
    while b:
        value = np.sum(crop<128) / np.sum(crop>128)
        print(value)
        if value > 5 or value == "inf":
            crop = skimage.exposure.adjust_gamma(crop, 0.5)
        elif value < 0.1:
            crop = skimage.exposure.adjust_gamma(crop, 1.5)
        else:
            b=False
    return crop

In [148]:
path = "./imgs/underexposed.jpg"
final = fix_image(imread(path),0)
if final is not None:
    plt.figure(figsize=(10,10))
    plt.subplot(1,2,1)
    plt.imshow(imread(path))
    plt.subplot(1,2,2)
    plt.imshow(final)


1/1 [==============================] - 0s 12ms/step
no faces
